## Preparing mass spec data for the open-cell website demo

In [ ]:
import os
import re
import sys
import glob
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors

In [ ]:
# targets for demo
# ['ATL2', 'ATL3', 'CLTA', 'LAMP1', 'MTOR', 'POLR1C', 'RAB14', 'SEC13', 'TAF12', 'VAPA',]

In [ ]:
filepath = '../demo-data/2019-08-02_volcano_plots_all.txt'
meta = pd.read_csv(filepath, sep='\t')

In [ ]:
meta_columns = [
    'Protein IDs',
    'Majority protein IDs',
    'Protein names',
    'Gene names',
    'Symbol color',
    'Symbol type',
    'Symbol size',
    'Show label',
    'Only identified by site',
    'Reverse',
    'Potential contaminant',
    'Peptides',
    'Razor + unique peptides',
    'Unique peptides',
    'Sequence coverage [%]',
    'Unique + razor sequence coverage [%]',
    'Unique sequence coverage [%]',
    'Mol. weight [kDa]',
    'Q-value',
    'Score',
    'Intensity',
    'MS/MS count',]

meta = meta[meta_columns].copy()
meta.rename(columns={col: col.lower().replace(' ', '_') for col in meta.columns}, inplace=True)

# create a gene_name column with the first name in the list of gene_names in each row
meta['gene_name'] = [str(s).split(';')[0] for s in meta.gene_names]

In [ ]:
# these columns appear to contain the gene name(s) and description for each hit,
# which are we currently need for the open-cell website demo
ms_metadata = meta[['gene_name', 'gene_names', 'protein_names']].copy()

In [ ]:
pvalue_pattern = r'^LFQi 1_(\w+): -Log\(P-value\)$'
enrichment_pattern = r'^LFQi 1_(\w+): Difference$'

In [ ]:
# create dataframes of pvalues and enrichments with columns names equal to gene_name
pvalue_columns = [col for col in d.columns if re.findall(pvalue_pattern, col)]
enrichment_columns = [col for col in d.columns if re.findall(enrichment_pattern, col)]

pvalues = d[pvalue_columns].copy()
pvalues.rename(columns={
    col: re.findall(pvalue_pattern, col)[0] for col in pvalues.columns}, inplace=True)

enrichments = d[enrichment_columns].copy()
enrichments.rename(columns={
    col: re.findall(enrichment_pattern, col)[0] for col in enrichments.columns}, inplace=True)

In [ ]:
enrichments.head()

## Calculating FDR curve parameters
This approach is an attempt to exactly implement the algorithm described in Hein 2015.

In [ ]:
def calc_fdr(x0, c):
    '''
    Calculate the false discovery rate (FDR) for a given x0 and c

    The FDR is the number of negatively-enriched hits with p-values above the FDR curve, 
    relative to the total number of hits with p-values above the FDR curve
    
    The FDR curve itself is a function only of enrichment and is given by
    c / (abs(enrichment) - x0)
    '''
    
    all_en = enrichments.values
    all_pval = pvalues.values
    
    pval_neg = all_pval[all_en < (-x0)]
    en_neg = all_en[all_en < (-x0)]
    num_neg = (pval_neg > (c / (-en_neg - x0))).sum()

    pval_pos = all_pval[all_en > x0]
    en_pos = all_en[all_en > x0]
    num_pos = (pval_pos > (c / en_pos - x0)).sum()
    
    return num_neg, num_pos

In [ ]:
# naive grid search over x0 and c
x0s, cs = np.meshgrid(np.arange(1.5, 5, .03), np.arange(1, 6, .03))

fdrs = x0s * 0
nums_true = x0s*0
for i in range(fdrs.shape[0]):
    for j in range(fdrs.shape[1]):
        num_neg, num_pos = calc_fdr(x0s[i][j], cs[i][j])
        fdrs[i][j] = num_neg / (num_neg + num_pos)
        nums_true[i][j] = num_pos

In [ ]:
# find the parameters with an FDR near the given threshold (usually 0.01 or 0.05)
thresh = 0.05
wiggle = .001

mask = (fdrs < (thresh + wiggle)) & (fdrs > (thresh - wiggle))
plt.imshow(mask)

In [ ]:
# find the parameters with the greatest number of positive hits
ind = np.argmax(nums_true[mask])
x0s[mask][ind], cs[mask][ind]

In [ ]:
# plot the FDR curve 
x0, c = 1.62, 4.25

x = np.arange(x0, 15, .1)
plt.plot(x, c/(x - x0), color='gray')

x = np.arange(-15, -x0, .1)
plt.plot(x, c/(-x - x0), color='gray')

plt.scatter(enrichments, pvalues, alpha=.1)
plt.gca().set_ylim([0, 30])

In [ ]:
num_neg, num_pos = calc_fdr(x0, c)
print('%d, %d, %0.4f' % (num_neg, num_pos, num_neg / (num_neg + num_pos)))

## Construct JSON data for demo

In [ ]:
# enrichments and p-values for all targets
ms_data = []
for target_name in pvals.columns:
    
    hits = []
    ms_datum = {'target_name': target_name}
    for ind, row in ms_metadata.iterrows():
        
        hits.append({
            'gene_id': ind,
            'pvalue': pvals[target_name].iloc[ind],
            'enrichment': enrichments[target_name].iloc[ind],
        })

    ms_datum['hits'] = hits
    ms_data.append(ms_datum)

In [ ]:
[row['target_name'] for row in ms_data]

In [ ]:
with open('../src/demo-data/ms_data.json', 'w') as file:
    json.dump(ms_data, file)
    
with open('../src/demo-data/ms_metadata.json', 'w') as file:
    ms_metadata.to_json(file, orient='index')